##### Processing and Cleaning the Data used in 
#### [Ease of Travel with your Passport (Tableau Viz)](https://public.tableau.com/app/profile/marul.s.nesamony/viz/Passport_Ranks_16914649641120/WherecanyourPassporttakeyouwithoutaVisa) 

1. Passport index of Countries and the country details from https://www.henleyglobal.com/passport-index/compare are stored in individual htm files in local and is looped to get the details for each country, which is then stored in cntry_df dataframe
2. Two reference files "Continents.csv" and "Country_Codes.htm" have the continents and country 3 digit abbreviations. The contents of these 2 reference files are stored in cntry_cd_df and continent_df dataframe
3. The continent details of the country and each of its corresponding visa-free country is added to the dataframe
4. The final dataframe contents are saved to a csv file to be consumed by the Tableau viz


In [ ]:
from bs4 import BeautifulSoup #for parsing HTML page
#import lxml
import csv #for writing into CSV file
import os
import pandas as pd
import numpy as np
#import urllib.request
#import piplite
#await piplite.install(['requests']) -- HTTPS not supported in JupyterLite using requests
#import requests

#print(os.getcwd()) #check to see what the current working directory is
os.chdir("/drive/notebooks/Training") #set the working directory

cntry_lst = []
cntry_code = [] 

i = 0

for files in os.listdir("/drive/notebooks/Training/Countries/"):
    #print(files)
    i +=1

    if files.endswith(".htm"): #and files not in ["County_Codes.htm"]:
        #print(files)
        with open("Countries/"+files) as country:
            soup = BeautifulSoup(country, "html.parser")

            #    soup
            if files not in ["Country_Codes.htm"]:
                t_rows = soup.find_all("div", class_="col-passport-info col-md-3")
                #t_rows = soup.find_all("body")
                for t_row in t_rows:
                    cntry_name = t_row.find("h3", class_="mb15").text    
                    cntry_rank = t_row.find_all("h3", class_="mb5")[0].text[:-2]
                    cntry_allowed = t_row.find_all("h3", class_="mb5")[1].text
                    flg_url = t_row.find("img").get("src")
                    #print(flg_url)
                    flg_name = flg_url[-6:]
                    flg_url = "https://raw.githubusercontent.com/marulsnesamony/CountryFlags/main/"+ flg_name
                    country_abbr = flg_name[:2].upper()
                    #print(cntry_name, country_abbr)
                    
                    visa_cntrys = t_row.find_all("div", class_="country-container")
                    #print(cntry_name, cntry_rank, cntry_allowed)
                    
                    for visa_cntry in visa_cntrys:
                        if len(visa_cntry) < 3: 
                            visa_free = 'N'
                        else: 
                            visa_free = 'Y'
                        visa_flg_url = visa_cntry.find("img").get("src")
                        visa_country_cd = visa_flg_url[-6:][:2].upper()
                        visa_cntry_nm = visa_cntry.find("p").text.rstrip("*").strip()
                        visa_cntry_len = len(visa_cntry)
                        
                        #print(visa_cntry_nm, visa_cntry_len, visa_country_cd)
                        # Append all the values to a List as Tuples - each append is a tuple
                        cntry_lst.append((cntry_name, flg_url, flg_name, country_abbr, cntry_rank, cntry_allowed, visa_cntry_nm, visa_country_cd, visa_free))
                    cntry_lst.append((cntry_name, flg_url, flg_name, country_abbr, cntry_rank, cntry_allowed, cntry_name, country_abbr, 'O'))
            else:
                t_rows = soup.find_all("tr")
                # column headers to be written to the csv file
                col_names = ['CountryName', 'Country_2_Digit_Abbr', 'Country_3_Digit_Abbr', 'Country_Code']

                # Create the CSV file to which the contents need to be written to
                # If the file exists, this will overwrite its contents else it will create the file 
                csv_file = open('/drive/notebooks/Training/Country_Codes.csv', 'w')
                csv_writer = csv.writer(csv_file)
                csv_writer.writerow(col_names)
                
                xtra_cntry = 0
                
                for t_row in t_rows:

                    if len(t_row) < 10:    #leave out table rows that do not have Country info
                        continue
                    # this will return everything within the text part of the 'td' tag
                    
                    row_len = len(t_row)
                    country_name = t_row.find_all("td")[1].text.strip()  
                    country_abbr_2 = t_row.find_all("td")[2].text.strip()
                    country_abbr_3 = t_row.find_all("td")[3].text.strip()
                    country_code = t_row.find_all("td")[4].text.strip()
                    if 'Kosova' in country_name:
                        xtra_cntry = 1
                    #print(country_name, row_len, country_abbr_2, country_abbr_3, country_code)
                    csv_writer.writerow([country_name, country_abbr_2, country_abbr_3, country_code])
                    cntry_code.append((country_name, country_abbr_2, country_abbr_3, country_code))
                else:
                    if xtra_cntry == 0:
                        country_name = 'Kosova'  
                        country_abbr_2 = 'XK'
                        country_abbr_3 = 'XXK'
                        country_code = 383
                        csv_writer.writerow([country_name, country_abbr_2, country_abbr_3, country_code])
                        cntry_code.append((country_name, country_abbr_2, country_abbr_3, country_code))                       
                    csv_file.close()

    
    elif files in ("Continents.csv"):
        #print(files)
        continent_df = pd.read_csv("Countries/"+files)
        continent_df["Code"] = np.where(continent_df["Entity"] == "Kosovo", "XXK", continent_df["Code"])
        
cntry_df = pd.DataFrame(cntry_lst, columns=('Country', 'Flag_URL', 'Flag_file', 'Country_Abbr', 'Country_Rank', 'Number_VisaFree', 'VisaFreeCountry', 'VisaFreeCntryCD', 'Visa_Free'))
#print(cntry_df)

cntry_cd_df = pd.DataFrame(cntry_code, columns=('CountryName', 'Country_2_Digit_Abbr', 'Country_3_Digit_Abbr', 'Country_Code'))
#print(cntry_cd_df)

countries = pd.merge(cntry_df, cntry_cd_df, left_on = 'Country_Abbr', right_on = 'Country_2_Digit_Abbr', how = 'left').drop(columns=['CountryName'])
#print(countries.describe())

country_cont_temp = pd.merge(countries, continent_df, left_on = 'Country_3_Digit_Abbr', right_on = 'Code', how = 'left').drop(columns=['Country_2_Digit_Abbr', 'Year'])

#Similar to listagg in Oracle
#visa_list = country_cont_temp.query('Visa_Free == "Y"').groupby(['Country_Abbr'])['VisaFreeCntryCD'].apply(','.join).reset_index()
visa_list = country_cont_temp.query('Visa_Free == "Y"').groupby(['Country_Abbr']).agg(Visa_Free_List = ('VisaFreeCntryCD', list), VisaFree_Number = ('VisaFreeCntryCD', 'count')).reset_index()
visa_list['Cntry_Rank'] = visa_list['VisaFree_Number'].rank(method='dense', ascending=False)
visa_list['Spiral_Rank'] = visa_list['VisaFree_Number'].rank(method='first', ascending=False)

#travel_from = country_cont.loc[country_cont["Visa_Free"] == "Y", ["Country_Abbr", "VisaFreeCntryCD"]].groupby(['VisaFreeCntryCD'])['Country_Abbr'].apply(','.join).reset_index()

country_cont_temp = pd.merge(country_cont_temp, visa_list, left_on = 'Country_Abbr', right_on = 'Country_Abbr', how = 'left').drop(columns=['Visa_Free_List'])
#country_cont.rename(columns={"VisaFreeCntryCD_x": "VisaFreeCntryCD", "VisaFreeCntryCD_y": "Visa_Free_List"}, inplace = True)

country_cont = pd.merge(country_cont_temp, visa_list, left_on = 'VisaFreeCntryCD', right_on = 'Country_Abbr', how = 'left', suffixes=('', '_y')).drop(columns=['Country_Abbr_y', 'VisaFree_Number_y', 'Cntry_Rank_y', 'Spiral_Rank_y'])
country_cont.rename(columns={"Visa_Free_List": "VisaFree_Travel_VF_List"}, inplace = True)
#country_cont.drop(columns=['Country_Abbr_y', 'VisaFree_Number_y', 'Cntry_Rank_y', 'Spiral_Rank_y'], inplace = True)

country_cont['VisaFree_To_Cntry'] = np.where(country_cont.apply(lambda x: str(x.Country_Abbr) in str(x.VisaFree_Travel_VF_List), axis=1) == True, 'Y', 'N')
country_cont.drop(columns=['VisaFree_Travel_VF_List'], inplace = True)

country_cont_temp_2 = pd.merge(country_cont, cntry_cd_df, left_on = 'VisaFreeCntryCD', right_on = 'Country_2_Digit_Abbr', how = 'left', suffixes=('', '_y'))#.drop(columns=['Country_2_Digit_Abbr_y', 'Country_Code_y'])
#country_cont_temp_2.rename(columns={"CountryName": "VisaFree_Country", "Country_3_Digit_Abbr_y": "VisaFree_3_Digit_CD"}, inplace = True)

country_cont = pd.merge(country_cont_temp_2, continent_df, left_on = 'Country_3_Digit_Abbr_y', right_on = 'Code', how = 'left', suffixes=('', '_z')).drop(columns=['Country_2_Digit_Abbr', 'Year'])
country_cont.drop(columns=['Country_Code_y', 'CountryName', 'Code_z', 'Entity_z', 'Code', 'Entity'], inplace = True)
country_cont.rename(columns={"Country_3_Digit_Abbr_y": "VisaFree_3_Digit_CD", "Continent_z": "VisaFree_Continent"}, inplace = True)
                           
os.chdir("/drive/notebooks/Training")
if(os.path.exists("/drive/notebooks/Training/Compare_Passport.csv") and os.path.isfile("/drive/notebooks/Training/Compare_Passport.csv")):
  os.remove("/drive/notebooks/Training/Compare_Passport.csv")
country_cont.to_csv("Compare_Passport.csv", mode='w', index=False)


##### **Code snippets for data validation**

In [ ]:
#find distinct countries for which there is no matching record in the second dataframe
#print(countries.query('Country_Abbr != Country_2_Digit_Abbr').Country.unique())
#print(countries.query('Country_Abbr == "IN"'))
print(country_cont.query('VisaFree_Travel_VF == False'))

In [ ]:
# find the list element that satisfies the criteria. This is a List Comprehension. 
# If the search criteria is a substring of the element in the list, it will show up as no match
print([i for i in cntry_code if 'Brunei' in i])

# find the dataframe record that satisfies the sub-string match criteria
print(cntry_cd_df.query('CountryName.str.contains("Brunei")'))
#print(help(cntry_code))

In [ ]:
# Describe the dataframe. # of records, Unique values of each column, freq = number of times the top value occurs in the column
#print(countries.describe())
#print(continent_df.head())
#print(cntry_cd_df.head())
#print(country_cont_temp_2.head(10))
print(country_cont.tail())
#print(visa_list.head())
#print(country_cont.describe(include='all'))
#print(visa_list.describe(include='all'))
#print(visa_list.query('Country_Abbr == "AF"'))
#print(visa_list['Number_VisaFree'].isnull())

In [ ]:
# Describe the dataframe when dataframe columns are mixed type. # of records, Unique values of each column, freq = number of times the top value occurs in the column
#print(country_cont.describe(include='all'))
#print(country_cont.query('Visa_Free == "Y"').groupby(['Country_Abbr'])['VisaFreeCntryCD'].apply(','.join))
print(country_cont.query('Visa_Free == "Y" and VisaFreeCntryCD == "MY"').Country_Abbr.unique())
print(country_cont.query('Visa_Free == "Y" and Country_Abbr == "MY"').VisaFreeCntryCD.unique())

In [ ]:
# Unique values of the CountryName column
#print(cntry_cd_df.CountryName.unique())
#print(np.where(country_cont['VisaFree_Travel_VF_List'].str.contains(str(country_cont['Country_Abbr'])), 'Y', 'N'))

print(np.where(country_cont.apply(lambda x: str(x.Country_Abbr) in str(x.VisaFree_Travel_VF_List), axis=1) == True, 'Y', 'N') )
print(country_cont.apply(lambda x: str(x.Country_Abbr) in str(x.VisaFree_Travel_VF_List), axis=1))
#print(str(country_cont['Country_Abbr']))
#print(country_cont['VisaFree_Travel_VF_List'].str.contains(str(country_cont['Country_Abbr'])) )
#print((str(country_cont['Country_Abbr'])) )
#print(str(country_cont['VisaFree_Travel_VF_List']) )

In [ ]:
# Find if the CountryName column in dataframe contains any element with the substring
print(cntry_cd_df['CountryName'].str.contains('osov').any())

In [ ]:
#print(continent_df['Entity'].str.contains('osov').any())
#print(continent_df.Entity.unique())
#print(country_cont.query('Country_3_Digit_Abbr != Code').Country.unique())
#print(continent_df.query('Entity.str.contains("osov")'))

#select specific columns from dataframe based on a condition
#print(country_cont.loc[country_cont["Visa_Free"] == "Y", ["Country_Abbr", "VisaFreeCntryCD"]])
print(country_cont.loc[country_cont["Visa_Free"] == "Y", ["Country_Abbr", "VisaFreeCntryCD"]].groupby(['VisaFreeCntryCD'])['Country_Abbr'].apply(','.join).reset_index())